In [1]:
import matplotlib.pyplot as plt

from build.autodiff import Var, Matrix, NeuralNetwork, computeMSELoss

In [2]:
in_dim = 1
out_dim = 1

N = 10

X = Matrix(N, in_dim)  # shape: (N, 1)
Y_labels = Matrix(N, out_dim)  # shape: (N, 1)

In [3]:
# Initialize training data
for i in range(N):
    X[i, 0] = Var(i)
    Y_labels[i, 0] = 5.0 * i + 3.0
    # y = 5x + 3

In [11]:
lr = 1e-2
epochs = 1000

model = NeuralNetwork([(in_dim, out_dim)])
model

NeuralNetwork =
[1 -> 1]

In [12]:
for epoch in range(epochs):
    # Reset gradients and the old graph
    X.resetGradAndParents()
    Y_labels.resetGradAndParents()

    for layer in model.layers:
        W, b = layer

        W.resetGradAndParents()
        b.resetGradAndParents()

    # Forward pass
    Y_pred = model.forward(X)

    # Calculate the loss
    loss = computeMSELoss(Y_labels, Y_pred)
    loss_val = loss.getVal()

    # Backpropagation (Reverse-Mode Automatic Differentiation)
    loss.setGrad(1.0)
    loss.backward()

    # Backpropagation and Gradient Descent for each parameter
    for layer in model.layers:
        W, b = layer

        # Update W
        for i in range(W.rows):
            for j in range(W.cols):
                weight_param = W[i, j]

                # Partial derivative of the Loss function with respect to the weight parameter
                gradient = weight_param.getGrad()
                weight_param.setVal(weight_param.getVal() - lr * gradient)

        # Update b
        for i in range(b.rows):
            for j in range(b.cols):
                bias_param = b[i, j]

                # Partial derivative of the Loss function with respect to the bias parameter
                gradient = bias_param.getGrad()
                bias_param.setVal(bias_param.getVal() - lr * gradient)

    if epoch % 100 == 0:
        print(f"Epoch: {epoch + 1} | Train Loss: {loss_val}")

Epoch: 1 | Train Loss: 855.9965644644811
Epoch: 101 | Train Loss: 0.4305055875552699
Epoch: 201 | Train Loss: 0.13869874231943552
Epoch: 301 | Train Loss: 0.04468546211034522
Epoch: 401 | Train Loss: 0.014396601516518037
Epoch: 501 | Train Loss: 0.004638245313735261
Epoch: 601 | Train Loss: 0.0014943331984082317
Epoch: 701 | Train Loss: 0.0004814388970010374
Epoch: 801 | Train Loss: 0.00015510825282638482
Epoch: 901 | Train Loss: 4.997221920523793e-05


In [15]:
# Make Predictions
Y_pred_final = model.forward(X)

print(f"Ground Truth Labels:\n{Y_labels}\n")
print(f"Final Model Predictions:\n{Y_pred_final}")

first_layer = model.layers[0]
W_learned, b_learned = first_layer

print(f"Learned W(0, 0) = {W_learned[0, 0]}")
print(f"Learned b(0, 0) = {b_learned[0, 0]}")

Ground Truth Labels:
Matrix(10 x 1) = 
3.000000 
8.000000 
13.000000 
18.000000 
23.000000 
28.000000 
33.000000 
38.000000 
43.000000 
48.000000 


Final Model Predictions:
Matrix(10 x 1) = 
2.992543 
7.993732 
12.994922 
17.996111 
22.997300 
27.998489 
32.999678 
38.000867 
43.002057 
48.003246 

Learned W(0, 0) = Var(val=5.001189, grad=0.000675)
Learned b(0, 0) = Var(val=2.992543, grad=-0.004235)


In [ ]:
from sklearn.datasets import make_moons, make_blobs

X, y = make_moons(n_samples=100, noise=0.1)

y = y * 2 - 1  # make y be -1 or 1

plt.figure(figsize=(5, 5))
plt.scatter(X[:, 0], X[:, 1], c=y, s=20, cmap="jet")